In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM

In [2]:
dc_evt = pd.read_csv('data/dc_evt_modified2.csv', low_memory=False)
dc_file = pd.read_csv('data/dc_file_modified2.csv', low_memory=False)

In [3]:
binary_col_file = [col for col in dc_file.columns if set(dc_file[col])|{0, 1} == {0, 1}]
binary_col_evt = [col for col in dc_evt.columns if set(dc_evt[col])|{0, 1} == {0, 1}]

In [4]:
bin_file_ag = dc_file[['inode', 'filename']+binary_col_file]
bin_file_ag['finode'] = bin_file_ag['inode'].astype(str) +"-"+ bin_file_ag["filename"]
bin_file_ag = bin_file_ag.drop(['inode'], axis=1)
bin_file_ag = bin_file_ag.drop(['filename'], axis=1)
bin_file_ag = bin_file_ag.groupby('finode').sum()

/tmp/ipykernel_39291/1880067057.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bin_file_ag['finode'] = bin_file_ag['inode'].astype(str) +"-"+ bin_file_ag["filename"]


In [50]:
model = OneClassSVM(kernel='rbf', nu=0.004)
model.fit(bin_file_ag)

OneClassSVM(nu=0.004)

In [51]:
y_pred = model.predict(bin_file_ag)
outliers = np.where(y_pred == -1)[0]
num_outliers = len(outliers)

In [52]:
outliers, num_outliers

(array([     0,      1,  10515,  10532,  10800,  10801,  10807,  10809,
         10873,  10946,  10958,  11005,  12164,  12276,  18401,  18446,
         18460,  34109,  36992,  87311,  87778,  87779,  87858,  88068,
        107762, 107791, 108026, 108029, 108045, 108046, 108047, 108048,
        108049, 108062, 108152, 108197, 108301, 109267, 110322, 110325,
        110326, 110330, 110427, 110430, 110458, 110482, 110502, 110519]),
 48)

In [72]:
with open('./one_class_svm_inodes.csv', 'w') as fp:
    tmp_df = bin_file_ag.copy()
    tmp_df.reset_index(inplace=True)
    tmp_df['inode'] = tmp_df['finode'].str.split('-', expand=True)[0].astype(int)
    tmp_df['filename'] = tmp_df['finode'].str.split('-', expand=True)[1]
    tmp_df.drop('finode', axis=1, inplace=True)
    tmp_df = tmp_df.reindex(columns=['inode', 'filename']+list(tmp_df.columns[:-2]))
    tmp_df.iloc[outliers].to_csv(fp)